In [16]:
import ollama
import psycopg2


In [90]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from sentence_transformers import SentenceTransformer


conn = psycopg2.connect(
    dbname="mydb",
    user="admin",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()


embedder = SentenceTransformer("BAAI/bge-m3")


def add_document(text):
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (content, embedding) VALUES (%s, %s)", (text, embedding))
    conn.commit()


base_url = "https://www.agnoshealth.com/forums/search?page="
page = 1

while True:
    url = base_url + str(page)
    web_data = requests.get(url)
    soup = BeautifulSoup(web_data.text, 'html.parser')

    
    find_word = soup.find_all("div", class_="flex flex-col")

  
    if not find_word:
        print("No data:", page)
        break

    for i in find_word:
        text = i.get_text(strip=True)
        print(text, "\n")
        add_document(text)

    page += 1
   


cur.close()
conn.close()

def generate_response(query_text):
    retrieved_docs = query_postgresql(query_text)
    context = "\n".join([doc[0] for doc in retrieved_docs])
    prompt = f"Answer the question:\n{context}\n\nQuestion:{query_text}"
    response = ollama.chat(model="llama3.2",messages=[
        {"role":"system","content":"You are a kind, informative AI medical assistant"},
        {"role":"user","content":prompt}
    ])
    return response["message"]["content"]

หญิง|อายุ23ปีเสี่ยงต่อโรคซึมเศร้า (Major depressive disorder)2/18/2024เสี่ยงต่อโรคซึมเศร้าอ่อนเพลียไม่มีแรง+3สวัสดีค่ะ  อยากจะปรึกษาค่ะในช่วง2-3วันที่ผ่านมามีความรู้สึกดาวน์มากๆ รู้สึกเบื่อ ไม่อยากทำอะไร ไม่อยากเจอใคร ใครพูดอะไรก็หงุดหงิดไปหมด บางครั้งอยู่ดีๆก็รู้สึกแย่จนร้องไห้ก็มี นอนไม่หลับด้วยค่ะ  กว่าจะหลับใช้เวลานานมาก  มาวันนี้ไม่รู้สึกดาวน์แล้วค่ะ  แต่มันเหนื่อย เพลีย  รู้สึกอยากนอนตลอดเวลา  นี่ไม่ใช่ครั้งแรกที่เป็นค่ะ  เป็นทุกครั้งที่เครียดมากๆ หรือมีความรู้สึกแย่ๆเข้ามา  กว่าจะหายต้องใช้เวลาเป็นอาทิตย์เลยค่ะ มันเป็นเพระเครียดมากจนเกินไปหรือเพราะอะไรคะ  แล้วควรไปพบแพทย์เพื่อรับการรักษาดีไหมคะ 

ชาย|อายุ21ปีกระเพาะปัสสาวะอักเสบ (Cystitis)2/16/2024กระเพาะปัสสาวะอักเสบปัสสาวะกะปริบกะปรอยปัสสาวะบ่อย+1สวัสดีครับคือว่าผมมีอาการนี้มาได้ปีสองปีแล้วครับ เคยรับการรักษาแล้วเหมือนได้ทานยาระงับการบีบตัวกระเพาะปัสสาวะอยู่ช่วงนึง แต่ไม่รู้สึกว่าดีขึ้นแล้วก็ไม่ได้ไปรับการรักษาอีกเลยจนมันชินไปเอง ปวดปัสสาวะบ่อยๆ แต่จำนวนไม่เยอะ แต่เหมือนตอนนี้เข้าห้องน้ำบ่อยๆ จะเป็นปัญหามากขึ้นแล้วกลัวว่าจะเป็

KeyboardInterrupt: 

In [74]:
conn = psycopg2.connect(
    dbname = "mydb",
    user = "admin",
    password = "1234",
    host = "localhost",
    port = "5432"
    
)
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
    id SERIAL PRIMARY KEY,
    content TEXT,
    embedding vector(1024)
""")
conn.commit()
cur.close( )
conn.close()


In [76]:
def query_postgresql(query_text, k=5):
    query_embedding = embedder.encode(query_text).tolist()
    conn = psycopg2.connect(
        dbname="mydb",
        user="admin",
        password="1234",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    query_embedding_str = "[" + ",".join(map(str, query_embedding)) + "]"
    sql_query = """
        SELECT content,embedding <=> %s::vector AS similarity_score
        FROM documents
        ORDER BY similarity_score ASC
        LIMIT %s;
    """
    cur.execute(sql_query, (query_embedding_str, k))
    results = cur.fetchall()
    conn.close()
    return results


query_postgresql("อาการของคนไข้อายุ 19")


[('หญิง|อายุ19ปีภาวะการปรับตัวผิดปกติ (Adjustment disorder)11/18/2023ภาวะการปรับตัวผิดปกติวิตกกังวลคิดมาก+2มีอาการเศร้าคิดมากเครียดวิตกกังวลหยุดคิดไม่ได้คิดซ้ำๆ ควรปรับตัวยังไงคะเพราะกลัวว่าจะกลับไปทำร้ายตัวเองอีก0แพทย์ตอบคำปรึกษาแล้วแชร์กระทู้',
  0.4102189051843953),
 ('หญิง|อายุ19ปีต่อมน้ำเหลืองที่คอติดเชื้อ (Cervical Lymphadenitis)7/4/2023เจ็บคอคันคอคอแดง+4มีก้อนขึ้นที่คอบริเวณ4จุด บริเวณท้ายทอยด้านขวา ข้างคอขวา ใต้คางและช่วงต่อระหว่างคอและไหล่ กดแล้วเจ็บ มีเลือดออกตามไรฟันตอนกลางคืนพอตื่นเช้ามาบ้วนน้ำลายแล้วมีสีเหลืองปนเลือดและมีีอาการเจ็บคอร่วมด้วยในทุกๆเช้า ส่วนตัวฟันซ้อนและมีฟันคุดล่างขวาที่ยังไม่ได้ถอนค่ะ ฟันใกล้ๆมีอาการปวดร่วมด้วยเวลาเคี้ยวอาหาร อาการอย่างอื่นนอกเหนือจากนี้ไม่มีค่ะ0แพทย์ตอบคำปรึกษาแล้วแชร์กระทู้',
  0.4326545676404713),
 ('หญิง|อายุ19ปีทอนซิลอักเสบ ติดเชื้อ (Bacterial pharyngotonsilitis)6/9/2023ทอนซิลอักเสบ ติดเชื้อเจ็บคอคันคอ+2เป็นทอมซิลอักเสบรอบที่ 3 ของปีค่ะ รอบเเรกเดือนกุมภา รอบสองเดือนพฤษภา รอบสามเดือนมิถุนาค่ะ ไม่กินเหล้า สูบบุหรี่ โรคประจำตัวคือ ภูมิแพ

In [89]:
def generate_response(query_text):
    retrieved_docs = query_postgresql(query_text)
    context = "\n".join([doc[0] for doc in retrieved_docs])
    prompt = f"Answer the question:\n{context}\n\nQuestion:{query_text}"
    response = ollama.chat(model="llama3.2",messages=[
        {"role":"system","content":"You are a professional and ethical medical doctor who provides accurate, safe, and easy-to-understand health advice to patients. When answering questions, you should use the patient’s medical records
"},
        {"role":"user","content":prompt}
    ])
    return response["message"]["content"]
                           
generate_response("อาการเลือดออกหลังมีเพศสัมพันธ์ เลือดสีน้ำตาล และเคยมีประวัติเลือดออกผิดปกติ มีความเสี่ยงอะไรบ้าง")

'ขออภัยครับ/ค่ะ แต่การให้คำปรึกษาเกี่ยวกับการมีเพศสัมพันธ์และอาการที่เกิดขึ้นหลังจากนั้นจะต้องอยู่ในความเชี่ยวชาญของแพทย์ผู้เชี่ยวชาญด้านเพศสภาพหรือแพทย์ผู้เชี่ยวชาญด้านสูติศาสตร์ ซึ่งมีความรู้และประสบการณ์ในเรื่องนี้มากขึ้น\n\nหากคุณมีอาการเลือดออกหลังมีเพศสัมพันธ์ และมีอาการเหล่านี้ เช่น เลือดสีน้ำตาล, การมีเพศสัมพันธ์กับบุคคลอื่นที่ไม่ได้รับการตรวจสอบว่าไม่มีโรคติดต่อทางเพศสัมพันธ์, หรือมีประวัติเลือดออกผิดปกติในอดีต มีความเสี่ยงในการติดเชื้อโรคต่างๆ เช่น โรคเรทรोवायरัส, โรคเฮปทิติสซี, โรคฮีวิทแมน-ดับเบิลยู (HIV)\n\nหากคุณกำลังพิจารณาการมีเพศสัมพันธ์ใหม่ ควรทำการตรวจสอบว่าบุคคลนั้นไม่มีโรคติดต่อทางเพศสัมพันธ์ และควรใช้ถุงยางป้องกันโรคติดต่อทางเพศสัมพันธ์ทุกครั้งที่มีเพศสัมพันธ์\n\nหากคุณมีอาการที่รุนแรงหรือไม่แน่ใจว่าเป็นอะไร ควรไปพบแพทย์ผู้เชี่ยวชาญด้านเพศสภาพหรือแพทย์ผู้เชี่ยวชาญด้านสูติศาสตร์เพื่อทำการตรวจสอบและให้คำปรึกษาอย่างเหมาะสมครับ/ค่ะ'